# vLife-Virtusa
## SAIL - Multiaccuracy
## Implementation of Multiaccuracy on EMR Data


#### Import Section

In [106]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, recall_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from t2dnn import ANN
from sklearn.linear_model import Ridge
seed=128

In [19]:
data=pd.read_csv('/home/ec2-user/SageMaker/vLife/Ruby/COHORT_SENSE_V1/balanced_data.csv')

In [20]:
data=data.rename(columns={'target':'target1'})

In [21]:
data['target2']=1-data['target1']

## Predictive Analysis
### Train Multiaccuracy Model
#### Train Test Split and initialize tf session

In [22]:
X_train,X_test,y_train,y_test=train_test_split(data.drop(['target1','target2'],axis=1),data[['target1','target2']],test_size=0.2,random_state=5)

In [23]:
inputX=X_train.values
inputY=y_train.values
inputX_test=X_test.values
inputY_test=y_test.values

In [24]:
learning_rate=0.001
training_epochs=2000
display_step=50
n_samples=inputY.shape[0]

In [37]:
net=ANN()
sess=tf.Session()
path='./model3/model.ckpt'
net.load_model(sess,path)

INFO:tensorflow:Restoring parameters from ./model3/model.ckpt
Model restored.


In [39]:
sess.run(net.cost,feed_dict={net.x:inputX,net.y_:inputY})

0.008591089

In [40]:
pred=sess.run(net.y,feed_dict={net.x:inputX_test})

In [42]:
control = tf.cast(tf.greater(net.y[:,1],net.y[:,0]), tf.float32)
noharm = [control, 1 - control, control + 1 - control]

In [43]:
logits = net.y[:,1] - net.y[:,0]

In [96]:
sess.run(logits,feed_dict={net.x:inputX_test,latent_ph:inputX_test})[:100]

array([-1.        , -0.99998665, -0.9999845 , -0.99999976, -0.99999267,
       -1.        , -0.80649817, -1.        ,  0.6456487 , -0.9999969 ,
        0.6390387 , -1.        , -1.        , -1.        ,  0.63763475,
        0.6344669 ,  0.6508851 ,  0.6420013 , -0.99996376,  0.6395769 ,
       -1.        , -1.        , -0.999996  , -0.99999595, -0.99974126,
       -1.        , -0.59469175, -1.        , -0.99999934,  0.6385342 ,
       -1.        ,  0.49974057,  0.64250267, -0.32492983, -1.        ,
       -1.        , -0.99999976,  0.6347921 , -0.9475297 ,  0.64054084,
       -1.        , -1.        ,  0.63920105,  0.6213882 ,  0.64490557,
        0.63680613,  0.6455529 , -0.9997397 , -1.        , -1.        ,
       -1.        , -0.99999875, -1.        , -1.        ,  0.6357789 ,
       -0.99999905, -1.        ,  0.6485051 ,  0.642035  ,  0.64290535,
        0.65638   ,  0.6375039 ,  0.6417881 , -0.99998087, -0.98518413,
        0.64293855,  0.6421031 , -0.9999997 , -1.        ,  0.62

In [44]:
max_T = 100
thresh = 1e-4

In [45]:
def sess_run(result, x, sess):
    num = x.shape[0]
    num_batch = np.ceil(num/200).astype(int)
    output = np.zeros(num)
    for batch in range(num_batch):
        output[batch*200:(batch+1)*200] = sess.run(result, feed_dict={net.x:inputX_test[batch*200:(batch+1)*200],latent_ph:inputX_test[batch*200:(batch+1)*200]})       
    return output

In [47]:
def res(p, y):
    return y * ((p>=0.1)/(p + 1e-20) + (p<0.1) * (20 - 100  * p)) +(1-y) * ((p < 0.9)/(1 - p + 1e-20) + (p>=0.9) * (100 * p - 80))

In [48]:
latent_ph = tf.placeholder(tf.float32, shape=(None, 124), name="latent_var")

In [49]:
## Train tensors using placeholders
best_epoch, best_acc = -1,0
coeffs = []
for t in range(max_T):
    control = tf.cast(tf.greater(net.y[:,1], net.y[:,0]), tf.float32)
    noharm = [control, 1 - control, control + 1 - control]
    print(logits)
    probs_heldout = sess_run(tf.nn.sigmoid(logits), inputX_test[800:1600], sess=sess)
    heldout_loss = np.mean(-inputY_test[:,0][800:1600] * np.log(probs_heldout + 1e-20) - (1-inputY_test[:,0][800:1600]) * np.log(1-probs_heldout + 1e-20))
    heldout_acc =  np.mean((probs_heldout>0.5)==inputY_test[:,0][800:1600])
    probs = sess_run(tf.nn.sigmoid(logits), inputX_test,sess)
    val_loss = np.mean(-inputY_test[:,0] * np.log(probs + 1e-20) - (1 - inputY_test[:,0]) * np.log(1 - probs + 1e-20))
    val_acc = np.mean((probs > 0.5) == inputY_test[:,0])
    if heldout_acc > best_acc:
        best_epoch = t
        best_acc = heldout_acc
        best_logits = logits
    delta = res(probs,inputY_test[:,0])
    residual = probs - inputY_test[:,0]
    for i, s in enumerate(noharm):
        temp_s = sess_run(noharm[i], inputX_test[:800], sess)
        temp_s_heldout = sess_run(noharm[i], inputX_test[800:1600], sess)
        samples1 = np.where(temp_s == 1)[0]
        samples2 = np.where(temp_s_heldout == 1)[0]
        clf = Ridge(alpha=1)
        #clf.fit(inputX_test[:800],inputY_test[:800])
        clf.fit(inputX_test[:800],delta[:800])
        clf_prediction = clf.predict(inputX_test[800:1600])
        #corr = np.mean(clf_prediction[:,0] * residual[800:1600])
        corr = np.mean(clf_prediction * residual[800:1600])
        print(t, i, corr)
        if corr > 1e-4:
            coeffs.append(clf.coef_)
            h = (tf.matmul(latent_ph, tf.constant(np.expand_dims(clf.coef_,-1),
                                                  dtype=tf.float32)) + clf.intercept_)
            h=tf.reshape(h,[-1])
            logits -= .1 * h * s
            break
    if i==2:
        break

Tensor("sub_3:0", shape=(?,), dtype=float32)
0 0 0.12297795177222579
Tensor("sub_6:0", shape=(?,), dtype=float32)
1 0 -0.04980155246352291
1 1 -0.04980155246352291
1 2 -0.04980155246352291


#### Define Groups for Training

In [30]:
races=['race_asian','race_black','race_hispanic','race_native','race_other','race_white']
for i in races:
    print(data[data[i]==1].shape)

(189, 126)
(706, 126)
(660, 126)
(51, 126)
(9, 126)
(6385, 126)


### Model Performance Metrics

In [115]:
probs = sess_run(net.y[:,1] - net.y[:,0], X_test, sess)
groups=['all','race_asian','race_black','race_hispanic','race_native','race_other','race_white']
errs = []
idxs = list(range(0,1600))
print('All_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('All_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_asian']==1))[0]
print('Asian_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('Asian_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_black']==1) )[0]
print('Black_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('Black_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_hispanic']==1))[0]
print('Hispanic_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('Hispanic_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_native']==1) )[0]
print('Native_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('Native_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_other']==1) )[0]
print('Other_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('Other_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_white']==1))[0]
print('White_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('White_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))

output = ''
dict1={}
for group, err in zip(groups, errs):
    dict1[group]=str(round(err, 1))
    output += group + ': ' + str(round(err, 1)) + ' & '
print('Original: ', output)


All_recall 0.9552795031055901 

All_precision 0.9424019607843137 

Asian_recall 0.9130434782608695 

Asian_precision 0.7777777777777778 

Black_recall 0.9425287356321839 

Black_precision 0.9647058823529412 

Hispanic_recall 0.9411764705882353 

Hispanic_precision 0.9302325581395349 

Native_recall 0.8333333333333334 

Native_precision 0.625 

Other_recall 0.0 

Other_precision 0.0 

White_recall 0.9619205298013245 

White_precision 0.9524590163934427 

Original:  all: 94.8 & race_asian: 78.4 & race_black: 93.8 & race_hispanic: 90.4 & race_native: 66.7 & race_other: 100.0 & race_white: 96.0 & 


In [64]:
pred[:50]

array([[1.0000000e+00, 3.4757381e-09],
       [9.9999332e-01, 6.6863627e-06],
       [9.9999225e-01, 7.7544291e-06],
       [9.9999988e-01, 9.6157699e-08],
       [9.9999630e-01, 3.6387466e-06],
       [1.0000000e+00, 1.3357912e-08],
       [9.0324908e-01, 9.6750937e-02],
       [1.0000000e+00, 1.5975912e-10],
       [2.4295757e-06, 9.9999762e-01],
       [9.9999845e-01, 1.5272209e-06],
       [2.7658317e-08, 1.0000000e+00],
       [1.0000000e+00, 4.5145145e-11],
       [1.0000000e+00, 5.4863593e-11],
       [1.0000000e+00, 7.0759745e-11],
       [2.0325375e-09, 1.0000000e+00],
       [1.6953173e-08, 1.0000000e+00],
       [1.1876276e-05, 9.9998808e-01],
       [4.0650942e-08, 1.0000000e+00],
       [9.9998188e-01, 1.8120552e-05],
       [4.3626944e-10, 1.0000000e+00],
       [1.0000000e+00, 3.8616825e-11],
       [1.0000000e+00, 3.9547830e-11],
       [9.9999797e-01, 1.9776669e-06],
       [9.9999797e-01, 2.0322805e-06],
       [9.9987066e-01, 1.2937262e-04],
       [1.0000000e+00, 8.

In [52]:
sess_run(tf.nn.sigmoid(best_logits), X_test, sess)

array([0.26894143, 0.26894405, 0.26894447, ..., 0.26894143, 0.26894143,
       0.73049074])

In [112]:
probs = sess_run(tf.nn.sigmoid(best_logits), X_test, sess)
groups=['all','race_asian','race_black','race_hispanic','race_native','race_other','race_white']
#groups = ['all', 'F', 'M', 'B', 'N', 'BF', 'BM', 'NF', 'NM']
errs = []
idxs = list(range(0,1600))
print('All_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('All_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_asian']==1))[0]
#print(idxs)
print('Asian_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('Asian_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_black']==1) )[0]
print('Black_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('Black_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_hispanic']==1))[0]
print('Hispanic_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('Hispanic_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_native']==1) )[0]
print('Native_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('Native_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_other']==1) )[0]
print('Other_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('Other_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
idxs = np.where((X_test['race_white']==1))[0]
print('White_recall',recall_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
print('White_precision',precision_score(y_test.iloc[idxs,0], (probs[idxs]<0.5)),'\n')
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
# idxs = np.where((X_test['race_white']==1) * (X_test['gender']==1))[0]
# errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
# idxs = np.where((X_test['race_white']==0) * (X_test['gender']==0))[0]
# errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
# idxs = np.where((X_test['race_white']==0) * (X_test['gender']==1))[0]
# errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test.iloc[idxs,0]))
output = ''
for group, err in zip(groups, errs):
    output += group + ': ' + str(round(err, 1)) + ' & '
print('MultiAccuracy Boost: ', output)

All_recall 0.9478260869565217 

All_precision 0.9490049751243781 

Asian_recall 0.9130434782608695 

Asian_precision 0.8076923076923077 

Black_recall 0.9425287356321839 

Black_precision 0.9647058823529412 

Hispanic_recall 0.9294117647058824 

Hispanic_precision 0.9404761904761905 

Native_recall 0.8333333333333334 

Native_precision 0.625 

Other_recall 0.0 

Other_precision 0.0 

White_recall 0.9536423841059603 

White_precision 0.9584026622296173 

MultiAccuracy Boost:  all: 94.8 & race_asian: 81.1 & race_black: 93.8 & race_hispanic: 90.4 & race_native: 66.7 & race_other: 100.0 & race_white: 95.9 & 


## END